## Deliverable 2. Create a Customer Travel Destinations Map.

In [118]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [119]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,73.06,47,0,1.99,clear sky
1,1,Nanortalik,GL,60.1432,-45.2371,9.14,61,100,13.76,overcast clouds
2,2,Cape Town,ZA,-33.9258,18.4232,70.12,70,0,1.01,clear sky
3,3,Champerico,GT,14.3000,-91.9167,81.36,64,27,1.23,scattered clouds
4,4,Inirida,CO,3.8653,-67.9239,74.14,94,79,1.83,broken clouds


In [95]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [96]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,73.06,47,0,1.99,clear sky
2,2,Cape Town,ZA,-33.9258,18.4232,70.12,70,0,1.01,clear sky
3,3,Champerico,GT,14.3000,-91.9167,81.36,64,27,1.23,scattered clouds
4,4,Inirida,CO,3.8653,-67.9239,74.14,94,79,1.83,broken clouds
5,5,Villarrica,PY,-25.7500,-56.4333,70.74,98,100,3.74,overcast clouds
14,14,Carnarvon,AU,-24.8667,113.6333,82.47,69,75,14.97,broken clouds
16,16,Kununurra,AU,-15.7667,128.7333,85.98,74,100,0.00,overcast clouds
17,17,Mitu,CO,1.1983,-70.1733,71.13,96,96,1.92,overcast clouds
18,18,Codrington,AU,-38.2667,141.9667,79.12,37,66,9.75,broken clouds
21,21,New Norfolk,AU,-42.7826,147.0587,72.72,39,0,1.99,clear sky


In [99]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                280
City                   280
Country                280
Lat                    280
Lng                    280
Max Temp               280
Humidity               280
Cloudiness             280
Wind Speed             280
Current Description    280
dtype: int64

In [100]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(inplace = True)


In [101]:
preferred_cities_df.count()

City_ID                280
City                   280
Country                280
Lat                    280
Lng                    280
Max Temp               280
Humidity               280
Cloudiness             280
Wind Speed             280
Current Description    280
dtype: int64

In [102]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,73.06,clear sky,-42.8794,147.3294,
2,Cape Town,ZA,70.12,clear sky,-33.9258,18.4232,
3,Champerico,GT,81.36,scattered clouds,14.3000,-91.9167,
4,Inirida,CO,74.14,broken clouds,3.8653,-67.9239,
5,Villarrica,PY,70.74,overcast clouds,-25.7500,-56.4333,
...,...,...,...,...,...,...,...
678,Tapiramuta,BR,65.37,overcast clouds,-11.8472,-40.7914,
680,Richards Bay,ZA,71.82,broken clouds,-28.7830,32.0377,
683,Noumea,NC,89.58,light rain,-22.2763,166.4572,
684,San Carlos,VE,79.21,overcast clouds,9.6612,-68.5827,


In [103]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [104]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,73.06,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
2,Cape Town,ZA,70.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
3,Champerico,GT,81.36,scattered clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
4,Inirida,CO,74.14,broken clouds,3.8653,-67.9239,Fuente del Guania Hotel de Lujo
5,Villarrica,PY,70.74,overcast clouds,-25.7500,-56.4333,Mansión Reljuv Tours SA
...,...,...,...,...,...,...,...
678,Tapiramuta,BR,65.37,overcast clouds,-11.8472,-40.7914,Casa Alencar
680,Richards Bay,ZA,71.82,broken clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
683,Noumea,NC,89.58,light rain,-22.2763,166.4572,Hôtel Le Lagon
684,San Carlos,VE,79.21,overcast clouds,9.6612,-68.5827,Hotel Paternopoli


In [105]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace = True)

In [109]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,73.06,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
2,Cape Town,ZA,70.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
3,Champerico,GT,81.36,scattered clouds,14.3000,-91.9167,"Hotel y restaurante ""El Diamante"""
4,Inirida,CO,74.14,broken clouds,3.8653,-67.9239,Fuente del Guania Hotel de Lujo
5,Villarrica,PY,70.74,overcast clouds,-25.7500,-56.4333,Mansión Reljuv Tours SA
...,...,...,...,...,...,...,...
678,Tapiramuta,BR,65.37,overcast clouds,-11.8472,-40.7914,Casa Alencar
680,Richards Bay,ZA,71.82,broken clouds,-28.7830,32.0377,BON Hotel Waterfront Richards Bay
683,Noumea,NC,89.58,light rain,-22.2763,166.4572,Hôtel Le Lagon
684,San Carlos,VE,79.21,overcast clouds,9.6612,-68.5827,Hotel Paternopoli


In [113]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [116]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [120]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))